In [ ]:
import pyamg
import numpy
import numpy as np
from scipy.io import loadmat
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from common import set_figure

Load some data

In [ ]:
#data = pyamg.gallery.load_example('airfoil')
data = pyamg.gallery.load_example('unit_square')

Create vertices and edges (for plotting)

In [ ]:
A = data['A'].tocsr()                              # matrix
V = data['vertices'][:A.shape[0]]                  # vertices of each variable
E = numpy.vstack((A.tocoo().row,A.tocoo().col)).T  # edges of the matrix graph

In [ ]:
def stencil_grid(S, grid):
    N_v = np.prod(grid)  # number of vertices in the mesh
    N_s = (S != 0).sum() # number of nonzero stencil entries

    # diagonal offsets
    diags = np.zeros(N_s, dtype=int)

    # compute index offset of each dof within the stencil
    strides = np.cumprod([1] + list(reversed(grid)))[:-1]
    indices = tuple(i.copy() for i in S.nonzero())
    for i, s in zip(indices, S.shape):
        i -= s // 2

    for stride, coords in zip(strides, reversed(indices)):
        diags += stride * coords

    data = S[S != 0].repeat(N_v).reshape(N_s, N_v)

    indices = np.vstack(indices).T

    # zero boundary connections
    for index, diag in zip(indices, data):
        diag = diag.reshape(grid)
        for n, i in enumerate(index):
            if i > 0:
                s = [slice(None)] * len(grid)
                s[n] = slice(0, i)
                s = tuple(s)
                diag[s] = 0
            elif i < 0:
                s = [slice(None)]*len(grid)
                s[n] = slice(i, None)
                s = tuple(s)
                diag[s] = 0

    # remove diagonals that lie outside matrix
    mask = abs(diags) < N_v
    if not mask.all():
        diags = diags[mask]
        data = data[mask]

    # sum duplicate diagonals
    if len(np.unique(diags)) != len(diags):
        new_diags = np.unique(diags)
        new_data = np.zeros((len(new_diags), data.shape[1]),
                            dtype=data.dtype)

        for dia, dat in zip(diags, data):
            n = np.searchsorted(new_diags, dia)
            new_data[n, :] += dat

        diags = new_diags
        data = new_data

    return sparse.dia_matrix((data, diags), shape=(N_v, N_v)).tocsr()

def poissonop(n, sten):
    #sten = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
    A = stencil_grid(sten, (n, n))
    return A

def getsten(epsilon):
    sten = np.array([[ 0      ,           -1.,  0],
                     [-epsilon,  2+2*epsilon, -epsilon],
                     [0       ,           -1,  0]])
    return sten

n = 128
epsilon = 0.001
sten = getsten(epsilon)
A = (n+1)**2 * poissonop(n, sten)

create an AMG hierarchy

In [ ]:
mls = pyamg.ruge_stuben_solver(A, max_levels=3, max_coarse=1,
                               CF='RS',keep=True)
print(mls)

In [ ]:
# The CF splitting, 1 == C-node and 0 == F-node
splitting = mls.levels[0].splitting
C_nodes = splitting == 1
F_nodes = splitting == 0

fig, ax = plt.subplots(figsize=(6,6))
ax.axis('equal')

for e in E:
    plt.plot(V[e,0], V[e,1], 'k-', zorder=1)
    
plt.scatter(V[:,0][C_nodes], V[:,1][C_nodes], c='r', s=100.0, zorder=2)  #plot C-nodes in red
plt.scatter(V[:,0][F_nodes], V[:,1][F_nodes], c='b', s=100.0, zorder=2)  #plot F-nodes in blue